Create a file with data from closest grid point
===

Import required modules and set file names
----

In [1]:
from netCDF4 import Dataset,date2num
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet

Variable metadata
---

In [2]:
# var = "tas"
# varTitle = "Closest-point temperature data"
# varLongName = "Reference height temperature"
# varUnits = "K"
var = "tasmax"
varTitle = "Closest-point temperature data"
varLongName = "Reference height temperature, daily max"
varUnits = "K"

Period-specific data
----

In [72]:
yr1 = "1986"
yr2 = "2015"
# yr1 = "1996"
# yr2 = "2005"

# mm = 6
# mm = 7
# mm = 8
mm = "JJA"
try:
    mms = "%02d" % mm
except TypeError:
    mms = mm

suff = yr1+"-"+yr2

File templates etc
---

In [73]:
model = "erai"
modelDataDir = "/Volumes/sbp1/model/pwrf/gis_%s/historical/wrf/postproc/%s" % (model, var)
# tasmax_wrf_erai_1996-2005_06_d.nc
modelDataFN = var+"_wrf_%s_%s_%s_d.nc" % ( model, suff, mms )

modelMetaFN = "wrf_geog.nc"
# modelMetaFN = "../erai/erai_geog_sub.nc"

newNCFN = "wrf_%s_%s_closest_%s_%s.nc" % ( model, var, suff, mms )

Load/define ERA Interim metadata
----

In [74]:
# modelMeta = Dataset( modelMetaFN, "r")
# lat = modelMeta.variables['lat'][:]
# ndim = len(lat.shape)
# lon = modelMeta.variables['lon'][:]

In [75]:
M = Model(modelMetaFN)
M.loadMeta()

minLon = -70 + 360
maxLon = -15 + 360
minLat = 58
maxLat = 86.5
M.setDomain( (minLat, maxLat, minLon, maxLon) )

2d dataset


Load AWS metadata
----

In [76]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

Find closest grid points
-----

In [77]:
C = []
for awsIX in (range(nSites)):
# for awsIX in ([0,1]):
# for awsIX in ([0]):
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    closest = M.closestPoints( awsLatLon )
    C.append( closest )
    if awsIX == 5:
        print closest.dtype
        print closest

[('ixLat', '<u2'), ('lat', '<f4'), ('ixLon', '<u2'), ('lon', '<f4'), ('distance', '<f8')]
[(113, 72.5353 , 101, 321.35077,  6.88726807)
 (114, 72.6704 , 101, 321.36145, 11.06182969)
 (113, 72.53159, 102, 321.80075, 11.47484497)
 (114, 72.66665, 102, 321.815  , 14.3741641 )]


Process data
----

In [78]:
print modelDataFN
D = Model(None, modelDataDir+"/"+modelDataFN)

time, timeCF = D.loadData( "time" )
print time[0], time[-1]
# dfTime = pd.Series( time, name="Time")

X = np.array( D.loadData( var ) )
nRec, nLat, nLon = X.shape
print nRec, nLat, nLon

Xnew = np.empty( (nRec, nSites), dtype = 'float' )

for awsIX in (range(nSites)):
# for awsIX in ((0,1)):
# for awsIX in ([5]):
    """ Basic AWS info """
    awsName = A.getName( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))

    closest = C[awsIX][0]
#     print awsLatLon, closest
#     print closest['lat'][0],closest['lon'][0]
    dx = "%.1f km" % closest[-1]
    print awsName, dx
    Xnew[:,awsIX] = X[:,closest['ixLat'],closest['ixLon']]

tasmax_wrf_erai_1986-2015_JJA_d.nc
1986-06-01 10:30:00 2015-08-31 10:30:00
2760 219 197
Swiss Camp 2.1 km
Crawford Point1 5.6 km
NASA-U 6.7 km
GITS 6.4 km
Humboldt 6.9 km
Summit 6.9 km
Tunu-N 4.2 km
DYE-2 3.8 km
JAR1 7.7 km
Saddle 5.6 km
South Dome 6.1 km
NASA-E 3.1 km
Crawford Point2 10.0 km
NGRIP 6.2 km
NASA-SE 7.2 km
KAR 1.2 km
JAR2 4.4 km
KULU 5.5 km
JAR3 1.0 km
Aurora 3.9 km
Petermann GL 6.5 km
Petermann ELA 4.9 km
NEEM 2.7 km


In [79]:
# Xnew[0:10,5]

Create a netCDF file for the new data
-----

In [80]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

ncfile = Dataset( newNCFN, mode='w', format='NETCDF4_CLASSIC' ) 
ncfile.title = varTitle
ncfile.period = suff

time_dim = ncfile.createDimension('time', None)
# ens_dim = ncfile.createDimension('ensemble', nEns)
stn_dim = ncfile.createDimension('station', nSites)

timeVar = ncfile.createVariable('time', np.float32, ('time',))
timeVar.units = timeCF.units
timeVar.calendar = timeCF.calendar
timeVar.standard_name = timeCF.standard_name
timeVar.long_name = timeCF.long_name
timeVar.axis = timeCF.axis

timeVar[:] = date2num(time, timeCF.units, timeCF.calendar)

# XVar = ncfile.createVariable(var, np.float32, ('time','station'))
# XVar = ncfile.createVariable(var, np.float32, ('ensemble','time','station'))
XVar = ncfile.createVariable(var, np.float32, ('time','station'))
XVar.units = varUnits
XVar.description = varTitle
XVar.long_name = varLongName
# XVar[:,:] = Xnew
XVar[:] = Xnew

ncfile.close()